In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.impute import SimpleImputer


In [30]:
data = pd.read_csv('diabetess.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [31]:
def outliers_base_group(column, replace=np.nan):
    Q1, Q3 = np.percentile(column, [25, 75])
    IQR = Q3 - Q1
    return column.where((column > (Q1 - 1.5 * IQR)) & (column < (Q3 + 1.5 * IQR)), replace)

columns_to_handle_nan = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction']

for method in ['mean', 'median', 'most_frequent']:
    imputer = SimpleImputer(strategy=method)
    for col in columns_to_handle_nan:
        data[f'{col}_{method}'] = data.groupby('Outcome')[col].transform(lambda x: outliers_base_group(x))

X = data.drop(['Outcome'], axis=1)
y = data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)


In [32]:
X = data_imputed.drop(['Outcome'], axis=1)
y = data_imputed['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)


In [33]:
model_unscaled = LogisticRegression().fit(X_train, y_train)
y_pred_unscaled = model_unscaled.predict(X_test)


/home/mohammad/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
scaler_std = StandardScaler()
X_standard_scaled = pd.DataFrame(scaler_std.fit_transform(X_train), columns=X_train.columns)
X_standard_test_scaled = pd.DataFrame(scaler_std.transform(X_test), columns=X_test.columns)
model_standard_scaled = LogisticRegression().fit(X_standard_scaled, y_train)
y_pred_standard_scaled = model_standard_scaled.predict(X_standard_test_scaled)


In [35]:
scaler_minmax = MinMaxScaler()
X_minmax_scaled = pd.DataFrame(scaler_minmax.fit_transform(X_train), columns=X_train.columns)
X_minmax_test_scaled = pd.DataFrame(scaler_minmax.transform(X_test), columns=X_test.columns)
model_minmax_scaled = LogisticRegression().fit(X_minmax_scaled, y_train)
y_pred_minmax_scaled = model_minmax_scaled.predict(X_minmax_test_scaled)


In [36]:
def eve(y_true, y_pred, label):
    accuracy = accuracy_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred)
    
    print(f"\n{label} Model:")
    print(f"Accuracy: {accuracy:.2%}")
    print("Confusion Matrix:")
    print(confusion)

In [37]:
eve(y_test, y_pred_unscaled, "Unscaled")


Unscaled Model:
Accuracy: 76.62%
Confusion Matrix:
[[131  20]
 [ 34  46]]


In [38]:
eve(y_test, y_pred_standard_scaled, "Standard Scaled")


Standard Scaled Model:
Accuracy: 77.06%
Confusion Matrix:
[[125  26]
 [ 27  53]]


In [39]:
eve(y_test, y_pred_minmax_scaled, "Min-Max Scaled")


Min-Max Scaled Model:
Accuracy: 77.06%
Confusion Matrix:
[[128  23]
 [ 30  50]]
